# Instructions
0. Install Biopython in Jupyter kernel if not already installed


Initilize PhyloTree object:

 1) assigns labels to internal nodes if labels are not present. 
    Default labels for internal nodes are of the form: IN_0_98,
    the first number is the node index and the second number its
    bootstrap value.
    
 2) (optional) collapses branches with bootstrap value below threshold

    2.1) Bootstrap values from fasttree are re-scaled from 0-1 to 0-100 to match
         iqtree values.
         
    2.2) Set bootstrap_threshold to None (default) to disable collapsing branches.

In [ ]:
from src import PhyloTree, filterFASTAbyIDs


"""
Arguments:
(optional) tree_format: defaults to 'newick'
(optional) bootstrap_threshold: defaults to None

NOTE:
Internal nodes are labelled at PhyloTree class initialization,
thus, labels may be different for different initialized PhyloTree
objects if a different bootstrap_threshold value is employ. This is
because if the threshold is set to anything other than 'None', then
branches with bootstrap value below threshold are collapsed, and some
internal nodes may disappear.
"""


tree_path = 'example.newick'

phylotree = PhyloTree(tree_path=tree_path, tree_format='newick', bootstrap_threshold=None)

In [3]:
# Export tree to file
phylotree.exportTree(outfile='labelled_example.newick', tree_format='newick')


Load 'tree.newick' into iTOL or tool of choice and select:

a) internal nodes of interest (copy label) and/or
b) leaves (final reference sequences) of interest

NOTE: iTOL removes '_' from labels, so, e.g. IN 1 9 in iTOL would be IN_1_9 here

Come back to jupyter notebook with initialized phylotree object.

EXAMPLE: we have selected some internal nodes and some leaves and
         are interested in finding out the resulting cluster in the
         first case, and the closest internal node (cluster) that 
         contains the selected leaves.


In [6]:
# First find out all leaves downstream of selected nodes
cluster_IN_3 = phylotree.getAllDescendantsOfTargetNode('IN_7_83')
cluster_IN_3

['ref_test_0', 'ref_test_14']

In [4]:
# First find out all leaves downstream of selected nodes
cluster_IN_5 = phylotree.getAllDescendantsOfTargetNode('IN_5_100')
cluster_IN_5

['ref_test_3', 'ref_test_5']

In [5]:
# Now find closest internal now to set of leaves

leaves = {
    'group_a': ['ref_test_14', 'ref_test_13'],
    'group_b': ['ref_test_3', 'ref_test_5']
}

In [6]:
IN_a = phylotree.getClosestCommonAncestor(target_names=leaves['group_a'])
IN_a

'IN_3_98'

In [7]:
# Now find closest internal now to set of leaves
IN_b = phylotree.getClosestCommonAncestor(target_names=leaves['group_b'])
IN_b

'IN_5_100'

In [8]:
"""
We can also enumerate all clusters as defined by internal nodes at once,
a python dictionary will be generated:
"""

tree_clusters = phylotree.extractClustersFromInternalNodes()

# All clusters are in this dictionary:
tree_clusters['IN_3_98']

['ref_test_3', 'ref_test_5', 'ref_test_0', 'ref_test_14', 'ref_test_13']

In [9]:
# Obtaining reference sequences of selected tree cluster

filterFASTAbyIDs(
    input_fasta="ref_database.faa",
    record_ids=tree_clusters["IN_3_98"],
    output_fasta="IN_3_98.faa"
)

In [10]:
tree_clusters

{'IN_0_None': ['ref_out_4',
  'ref_out_0',
  'ref_out_2',
  'ref_out_3',
  'ref_out_1',
  'ref_test_3',
  'ref_test_5',
  'ref_test_0',
  'ref_test_14',
  'ref_test_13',
  'ref_test_1',
  'ref_test_7',
  'ref_test_15',
  'ref_test_9',
  'ref_test_11',
  'ref_test_6',
  'ref_test_8',
  'ref_test_4',
  'ref_test_16',
  'ref_test_12',
  'ref_test_2',
  'ref_test_10'],
 'IN_1_100': ['ref_out_2', 'ref_out_3'],
 'IN_2_100': ['ref_test_3',
  'ref_test_5',
  'ref_test_0',
  'ref_test_14',
  'ref_test_13',
  'ref_test_1',
  'ref_test_7',
  'ref_test_15',
  'ref_test_9',
  'ref_test_11',
  'ref_test_6',
  'ref_test_8',
  'ref_test_4',
  'ref_test_16',
  'ref_test_12',
  'ref_test_2',
  'ref_test_10'],
 'IN_3_98': ['ref_test_3',
  'ref_test_5',
  'ref_test_0',
  'ref_test_14',
  'ref_test_13'],
 'IN_4_100': ['ref_test_3', 'ref_test_5', 'ref_test_0', 'ref_test_14'],
 'IN_5_100': ['ref_test_3', 'ref_test_5'],
 'IN_6_98': ['ref_test_1',
  'ref_test_7',
  'ref_test_15',
  'ref_test_9',
  'ref_test_11